In [1]:
import qctoolkit as qtk
import numpy as np
import itertools

/home/samio/local/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Test Gaussian basis set compactibility

In [2]:
with open('gbasis_ano_H-Xe.txt') as f:
    ano = f.readlines()

qmsetting = dict(
    program='gaussian',
    theory='ccsd(t)',
)

basis_list = [['Def2-TZVPP', 'def2t'], ['Def2-QZVPP', 'def2q'], [ano, 'ano']]

inps_grp = []
for basis, bname in basis_list:
    inps = []
    inps_grp.append(inps)
    for mol_str in ['FF', 'ClCl', 'BrBr', 'II']:
        mol = qtk.Molecule(mol_str)
        mol.name = '%02d%02d_%s_%s' % (mol.Z[0], mol.Z[1], mol.name, bname)
        inp = qtk.QMInp(mol, basis_set=basis, **qmsetting)
        inps.append(inp)

In [3]:
inps_halogen = list(qtk.flatten(inps_grp))

In [4]:
qtk.qmWriteAll(inps_halogen, 'halogenTest', overwrite=True, file_per_folder=2000)

# Scan diatomics

In [5]:
Z_range = range(1, 55)
Z_list = [[Z, Z] for Z in Z_range] + list(itertools.combinations(Z_range, 2))
# rdkit smiles is not working for exotic molecules
#smiles_list = map(lambda x: str(qtk.Z2n(x[0])) + str(qtk.Z2n(x[1])), Z_list)

In [6]:
inps_grp = []
for basis, bname in basis_list[:2]:
    inps = []
    inps_grp.append(inps)
    for Z1, Z2 in Z_list:
        R1 = qtk.element[Z1].atmrad
        R2 = qtk.element[Z2].atmrad
        R = (R1 + R2) / 2
        for R_factor in np.linspace(0.6, 3.0, 10, endpoint=True):
            d = R_factor * R
            f_str = ('%0.2f' % R_factor).replace('.', 'p')
            m = 1 if (Z1 + Z2) % 2 == 0 else 2
            for duplet in [0, 2]:
                mol = qtk.Molecule()
                mol.build([[Z1, d, 0, 0], [Z2, 0, 0, 0]])
                mol.setChargeMultiplicity(0, m + duplet)
                mol.name = '%02d%02d_%s_d%s_m%d' % (mol.Z[0], mol.Z[1], mol.name, f_str, m + duplet)
                inp = qtk.QMInp(mol, basis_set=basis, **qmsetting)
                inps.append(inp)

In [9]:
for i, (basis, bname) in enumerate(basis_list[:2]):
    inps = inps_grp[i]
    qtk.qmWriteAll(inps, 'Xe2Scan_%s' % bname, overwrite=True, file_per_folder=500, compress=True)

overwrite existing folder Xe2Scan_def2t
qmWriteAll: compression completed 
overwrite existing folder Xe2Scan_def2q
qmWriteAll: compression completed 
